In [ ]:
using AlphaVantage

In [ ]:
using DataFrames
tickers = ["B3SA3.SA", "PETR4.SA", "VALE3.SA", "ITUB4.SA", "BBAS3.SA", "USIM5.SA"]

returns_df = DataFrame()
for ticker in tickers
    df = DataFrame(time_series_daily(ticker, outputsize="full"))
    # limit the size of df to be the same as the others (to avoid errors)
    df = df[1:1000, :]
    returns = df.close[2:end] ./ df.close[1:end-1] .- 1
    returns_df[!, Symbol(ticker)] = returns
end

In [ ]:
returns_df

In [ ]:
using CSV
CSV.write("returns.csv", returns_df)

In [ ]:
new_df = CSV.read("./returns.csv", DataFrame)


In [30]:
using Dualization, JuMP, HiGHS

σ = [100,200,300]
r_bar = [1,2,3]
Gamma = 5000



model = Model()
@variable(model, r[1:3] >= 0)
@variable(model, z[1:3] >= 0)
@constraint(model, negcon[i = 1:3], - r[i] + z[i]*σ[i] >= -r_bar[i])
@constraint(model, poscon[i = 1:3],  r[i] + z[i]*σ[i] >= r_bar[i])
@constraint(model, equalcon, sum(z) == Gamma)
@constraint(model, lesscon[i = 1:3], z[i] ≤ 1)
@objective(model, Min, sum(r .* r_bar))

r[1] + 2 r[2] + 3 r[3]

In [31]:
dual_model = dualize(model, dual_names = DualNames("dual_", ""))

A JuMP Model
Maximization problem with:
Variables: 10
Objective function type: AffExpr
`AffExpr`-in-`MathOptInterface.GreaterThan{Float64}`: 6 constraints
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 6 constraints
`VariableRef`-in-`MathOptInterface.LessThan{Float64}`: 3 constraints
Model mode: AUTOMATIC
CachingOptimizer state: NO_OPTIMIZER
Solver name: No optimizer attached.
Names registered in the model: dual_equalcon_1, dual_lesscon[1]_1, dual_lesscon[2]_1, dual_lesscon[3]_1, dual_negcon[1]_1, dual_negcon[2]_1, dual_negcon[3]_1, dual_poscon[1]_1, dual_poscon[2]_1, dual_poscon[3]_1, r[1], r[2], r[3], z[1], z[2], z[3]

In [26]:
print(model)

In [32]:
print(dual_model)